# Model Klasifikasi Multi-Kelas Logistic Regression untuk Prediksi Penyakit Jantung

Notebook ini menunjukkan implementasi **Logistic Regression** menggunakan pustaka `scikit-learn` dan `mglearn` untuk memprediksi tahap penyakit jantung berdasarkan fitur yang tersedia.

Model ini akan dievaluasi menggunakan metrik berikut:

- **Akurasi**
- **Presisi (macro & weighted)**
- **Recall (macro & weighted)**
- **F1-Score (macro & weighted)**
- **AUC (Area Under Curve) untuk setiap kelas**
- **ROC (Receiver Operating Characteristic Curve) multi-kelas**

Selain itu, setiap rumus yang digunakan akan dijelaskan secara matematis.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import mglearn

# Mengatur gaya tampilan grafik
sns.set_style("whitegrid")


## 1. Memuat dan Menampilkan Dataset

Kita akan memuat dataset dan melihat struktur datanya.


In [ ]:
df = pd.read_csv("/mnt/data/HeartDisease.csv")

## 2. Pra-pemrosesan Data

Agar model dapat bekerja dengan baik, kita melakukan beberapa langkah berikut:
- Menghapus nilai yang hilang (NaN)


In [ ]:
# Menghapus nilai NaN jika ada
df.dropna(inplace=True)

# Menampilkan jumlah data setelah pembersihan
df.info()


## 3. Memisahkan Data Latih dan Data Uji

Kita akan membagi dataset menjadi **data latih (80%)** dan **data uji (20%)**.


In [ ]:
# Mendefinisikan fitur dan target untuk multi-kelas
X = df.drop(columns=["target"])
y = df["target"]

# Membagi dataset menjadi 80% latih dan 20% uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Menampilkan jumlah data per kelas setelah pembagian
print("Distribusi kelas dalam data latih:\n", y_train.value_counts())
print("Distribusi kelas dalam data uji:\n", y_test.value_counts())


## 4. Melatih Model Logistic Regression

Model logistic regression akan dipelajari berdasarkan data latih yang telah dibagi sebelumnya.


In [ ]:
# Inisialisasi dan pelatihan model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Menampilkan koefisien model
print("Koefisien model:", model.coef_)
print("Intercept model:", model.intercept_)


## 5. Melakukan Prediksi

Setelah model dilatih, kita akan menggunakannya untuk memprediksi data uji.


In [ ]:
# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan 5 hasil prediksi pertama
print("Prediksi pertama:", y_pred[:5])


## 6. Evaluasi Model

Kita akan mengevaluasi model menggunakan enam metrik utama:

### 6.1 Akurasi
\[ Accuracy = \frac{TP + TN}{TP + TN + FP + FN} \]
Akurasi menunjukkan seberapa sering model membuat prediksi yang benar.

### 6.2 Presisi
\[ Precision = \frac{TP}{TP + FP} \]
Presisi mengukur seberapa banyak prediksi positif yang benar dibandingkan dengan semua prediksi positif.

### 6.3 Recall
\[ Recall = \frac{TP}{TP + FN} \]
Recall mengukur seberapa banyak sampel positif yang benar-benar terdeteksi oleh model.

### 6.4 F1-Score
\[ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall} \]
F1-Score adalah rata-rata harmonik antara presisi dan recall.

### 6.5 AUC (Area Under Curve)
AUC mengukur seberapa baik model membedakan antara kelas positif dan negatif.

### 6.6 ROC (Receiver Operating Characteristic Curve)
ROC adalah kurva yang menunjukkan hubungan antara True Positive Rate (TPR) dan False Positive Rate (FPR).


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report

# Melakukan prediksi probabilitas untuk setiap kelas
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

# Binarisasi target untuk multi-class ROC
classes = np.unique(y_test)
y_test_binarized = label_binarize(y_test, classes=classes)

# Menghitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average="macro")
recall_macro = recall_score(y_test, y_pred, average="macro")
f1_macro = f1_score(y_test, y_pred, average="macro")
auc_multi = roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovo')

# Menampilkan hasil evaluasi
print(f"Akurasi: {accuracy:.3f}")
print(f"Presisi (Macro): {precision_macro:.3f}")
print(f"Recall (Macro): {recall_macro:.3f}")
print(f"F1-Score (Macro): {f1_macro:.3f}")
print(f"AUC Multi-Class: {auc_multi:.3f}")

# Menampilkan laporan klasifikasi lengkap
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

# Plot ROC Curve untuk setiap kelas
plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    plt.plot(fpr, tpr, label=f"Class {classes[i]} (AUC = {roc_auc_score(y_test_binarized[:, i], y_pred_proba[:, i]):.3f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Multi-Class")
plt.legend()
plt.show()
